In [1]:
!pip install pyspark -q
!pip install findspark -q

In [2]:
# Import the cleaned dataset (gold layer)
from google.colab import drive
drive.mount('/content/drive')

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("LoadGold").getOrCreate()

df = (spark.read
          .option("header", True)
          .option("inferSchema", True)
          .option("sep", ";")
          .csv('/content/drive/MyDrive/datasets/gold_fact_events.csv')
)

Mounted at /content/drive


In [3]:
from pyspark.sql import functions as F

feats = df \
  .withColumn("hour",   F.hour("event_ts")) \
  .withColumn("day",    F.dayofmonth("event_ts")) \
  .withColumn("month",  F.month("event_ts")) \
  .withColumn("year",   F.year("event_ts")) \
  .withColumn("weekday",F.date_format("event_ts","E"))

feats.select("event_ts","hour","weekday").show(5, False)


+-------------------+----+-------+
|event_ts           |hour|weekday|
+-------------------+----+-------+
|2015-05-03 03:00:04|3   |Sun    |
|2015-05-03 03:00:11|3   |Sun    |
|2015-05-03 03:00:13|3   |Sun    |
|2015-05-03 03:00:24|3   |Sun    |
|2015-05-03 03:00:26|3   |Sun    |
+-------------------+----+-------+
only showing top 5 rows



In [4]:
# Feature by items
item_features = (
    df
    .groupBy("Item_id")
    .pivot("event_type",["view", "addtocart", "transaction"])
    .count()
    .na.fill(0)
    .withColumn("ctr", F.when(F.col("view")>0, F.col("transaction")/F.col("view")).otherwise(0))
)
item_features.show(10)
item_features.printSchema()

+-------+----+---------+-----------+-------------------+
|Item_id|view|addtocart|transaction|                ctr|
+-------+----+---------+-----------+-------------------+
| 128367|  35|        0|          0|                0.0|
| 212010|   4|        0|          0|                0.0|
| 195577|   3|        0|          0|                0.0|
|  38311|  44|        1|          0|                0.0|
| 287536|  27|        6|          2|0.07407407407407407|
| 230443|   3|        0|          0|                0.0|
| 237019| 108|        7|          6|0.05555555555555555|
| 322355|  61|        7|          0|                0.0|
| 441201|   2|        0|          0|                0.0|
| 264914|   2|        0|          0|                0.0|
+-------+----+---------+-----------+-------------------+
only showing top 10 rows

root
 |-- Item_id: integer (nullable = true)
 |-- view: long (nullable = true)
 |-- addtocart: long (nullable = true)
 |-- transaction: long (nullable = true)
 |-- ctr: double 

In [5]:
# Features by visitor
user_features = (
   df
    .groupBy("visitor_id")
    .pivot("event_type", ["view","addtocart","transaction"])
    .count()
    .na.fill(0)
    .withColumn("conv_rate", F.when(F.col("view")>0, F.col("transaction")/F.col("view")).otherwise(0))
)

user_features.show(10)


+----------+----+---------+-----------+---------+
|visitor_id|view|addtocart|transaction|conv_rate|
+----------+----+---------+-----------+---------+
|   1191565|   1|        0|          0|      0.0|
|   1288720|   1|        0|          0|      0.0|
|    167316|   1|        0|          0|      0.0|
|   1308839|   1|        0|          0|      0.0|
|    195577|   1|        0|          0|      0.0|
|    150051|  27|        7|          0|      0.0|
|    807527|   5|        0|          0|      0.0|
|   1098309|   1|        0|          0|      0.0|
|    837609|   1|        0|          0|      0.0|
|    620588|   1|        0|          0|      0.0|
+----------+----+---------+-----------+---------+
only showing top 10 rows



In [6]:
from pyspark.sql.window import Window
from pyspark.sql.functions import lag, unix_timestamp, sum, when

w = Window.partitionBy("visitor_id").orderBy("event_ts")

session_df = (
  df
    .withColumn("prev_ts", lag("event_ts").over(w))
    .withColumn(
      "gap_min",
      (unix_timestamp("event_ts") - unix_timestamp("prev_ts"))/60
    )
    .withColumn(
      "new_session",
      when(F.col("gap_min").isNull() | (F.col("gap_min")>30), 1).otherwise(0)
    )
    .withColumn(
      "session_id",
      sum("new_session").over(w)
    )
)

# Calcule quelques métriques par session
session_feats = (
  session_df
    .groupBy("visitor_id","session_id")
    .agg(
      F.count("*").alias("session_events"),
      F.min("event_ts").alias("session_start"),
      F.max("event_ts").alias("session_end"),
      (unix_timestamp(F.max("event_ts")) - unix_timestamp(F.min("event_ts"))).alias("session_duration_s")
    )
)

session_feats.show(5, False)


+----------+----------+--------------+-------------------+-------------------+------------------+
|visitor_id|session_id|session_events|session_start      |session_end        |session_duration_s|
+----------+----------+--------------+-------------------+-------------------+------------------+
|1         |1         |1             |2015-08-13 17:46:06|2015-08-13 17:46:06|0                 |
|3         |1         |1             |2015-08-01 07:10:35|2015-08-01 07:10:35|0                 |
|5         |1         |1             |2015-07-17 01:45:56|2015-07-17 01:45:56|0                 |
|6         |1         |1             |2015-08-30 06:03:48|2015-08-30 06:03:48|0                 |
|6         |2         |4             |2015-08-30 06:39:38|2015-08-30 06:56:33|1015              |
+----------+----------+--------------+-------------------+-------------------+------------------+
only showing top 5 rows



In [7]:
from pyspark.sql import functions as F

events = df \
    .dropDuplicates(["event_ts","visitor_id","item_id"]) \
    .withColumn("hour",    F.hour("event_ts")) \
    .withColumn("weekday", F.date_format("event_ts","E"))

it = (
    item_features
      .withColumnRenamed("view",        "item_view")
      .withColumnRenamed("addtocart",   "item_addtocart")
      .withColumnRenamed("transaction", "item_transaction")
      .withColumnRenamed("ctr",         "item_ctr")
)

us = (
    user_features
      .withColumnRenamed("view",         "user_view")
      .withColumnRenamed("addtocart",    "user_addtocart")
      .withColumnRenamed("transaction",  "user_transaction")
      .withColumnRenamed("conv_rate",    "user_conv_rate")
)

train_df = (
    events.alias("e")
      .join(it.alias("it"), ["item_id"],    "left")
      .join(us.alias("us"), ["visitor_id"], "left")
)

# 5. Aperçu
train_df.select(
    "event_ts","hour","weekday",
    "visitor_id","user_view","user_conv_rate",
    "item_id","item_view","item_ctr"
).show(5, truncate=False)


+-------------------+----+-------+----------+---------+--------------+-------+---------+-------------------+
|event_ts           |hour|weekday|visitor_id|user_view|user_conv_rate|item_id|item_view|item_ctr           |
+-------------------+----+-------+----------+---------+--------------+-------+---------+-------------------+
|2015-05-03 03:41:35|3   |Sun    |411745    |47       |0.0           |408080 |6        |0.0                |
|2015-05-03 03:17:08|3   |Sun    |726427    |5        |0.0           |30155  |53       |0.05660377358490566|
|2015-05-03 03:00:27|3   |Sun    |1149227   |1        |0.0           |29757  |222      |0.04054054054054054|
|2015-05-03 03:52:59|3   |Sun    |1339015   |1        |0.0           |262521 |200      |0.01               |
|2015-05-03 03:04:28|3   |Sun    |1361384   |1        |0.0           |228525 |88       |0.0                |
+-------------------+----+-------+----------+---------+--------------+-------+---------+-------------------+
only showing top 5 